In [1]:
from google.colab import drive
drive.mount('/content/drive')
# Change accordingly
%cd /content/drive/MyDrive/project

Mounted at /content/drive
/content/drive/MyDrive/project


In [2]:
!pip install datasets
!pip install bllipparser
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 554.1/554.1 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for bllipparser: filename=bllipparser-2021.11.7-cp310-cp310-linux_x86_64.whl size=7369682 sha256=3402de4c6c0982d9c814d74f99c7214c94002bfde6ad4d2e3e088726103c0f9c
  Stored in directory: /root/.cache/pip/wheels/51/44/ac/c1bf9ca5f2c5dec6a44220ddb1aa712d127c0d4de5247a1eec
Successfully built bllipparser
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 13.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a

In [3]:
import os
import random
import json
import nltk
from datasets import load_dataset
from nltk.data import find
from nltk.tokenize import word_tokenize
from bllipparser import RerankingParser
import spacy
from tqdm import tqdm
from nltk import pos_tag

nltk.download('bllip_wsj_no_aux')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package bllip_wsj_no_aux to /root/nltk_data...
[nltk_data]   Unzipping models/bllip_wsj_no_aux.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
# def get_augment_sentence(sentence, ooc_words, parser):
#     # Just get the augmented english sentences first, get translation using baseline separately then combine
#     best_sentence = None
#     best_score = 0
#     # Tokenize input sentence
#     words = word_tokenize(sentence)

#     for pos in range(len(words)):
#         for ooc_word in ooc_words:
#             temp_words = words.copy()
#             temp_words[pos] = ooc_word
#             sentence_with_word_replaced_with_ooc_word = " ".join(temp_words)
#             nbest_list = parser.parse(sentence_with_word_replaced_with_ooc_word)

#             if len(nbest_list) > 0:
#                 # Get the best parse according to the reranker
#                 best_parse = nbest_list.get_reranker_best()
#                 reranker_score = abs(best_parse.reranker_score)
#                 # Update the best sentence if this parse is better than the current best
#                 if reranker_score > best_score:
#                     best_score = reranker_score
#                     best_sentence = sentence_with_word_replaced_with_ooc_word

#     return best_sentence

In [ ]:
# def augment_dataset_and_save(ooc_words, num_sentences_to_augment=10):
#     data_dir = "./data"
#     splits = ["train"]
#     dataset = load_dataset("iwslt2017", "iwslt2017-en-zh", data_dir=data_dir, cache_dir=data_dir)

#     # Initialize the translator
#     model_dir = find('models/bllip_wsj_no_aux').path
#     parser = RerankingParser.from_unified_model_dir(model_dir)

#     for split in splits:
#         augmented_sentences = []
#         indices = list(range(len(dataset[split])))

#         # Randomly choose sentences to do OOC word replacement
#         random_indices = random.sample(indices, min(num_sentences_to_augment, len(indices)))

#         for i in tqdm(random_indices):
#             example = dataset[split][i]
#             source_sentence = example['translation']['en']
#             target_sentence = example['translation']['zh']

#             augmented = get_augment_sentence(source_sentence, ooc_words, parser)
#             if augmented:
#                 augmented_sentences.append(augmented)

#         path = os.path.join(data_dir, f"{split}_parse_augmented_en.txt")
#         with open(path, 'w', encoding='utf-8') as f:
#             for s in augmented_sentences:
#                 f.write(s + "\n")

In [ ]:
with open('data/OOC words.txt', 'r') as f:
    # Read each line and create a set of strings
    ooc_words = {line.strip() for line in f}
print(ooc_words)

{'compensable', 'indecency', 'enumerate', 'concurrent', 'preclude', 'cupcake', 'enlistment', 'fluoride', 'snitch', 'Overcrowding', 'armour', 'offence', 'stipulates', 'incheon', 'colour', 'counselling', 'mutiny', 'scoundrel', 'prepare', 'integer', 'neighbouring', 'favour', 'paramedic', 'liaison', 'avenge', 'pawn', 'commonwealth', 'humour', 'endeavours', 'invalid', 'objecting', 'heartfelt', 'behaviour', 'aeroplane', 'supremacy', 'unlawful', 'incompetence', 'reaffirms', 'wrench', 'programmes', 'doubtful', 'valiant', 'fugitives', 'honourable', 'dampen', 'airspace', 'harmonise', 'enrolment', 'chairperson', 'drool'}


In [ ]:
# Use pretrained baseline ft.py script to get translations, save into {split}_parse_augmented_zh.json
# Combine into one single json with format
#'translation': {
#                 'en': aug_source,
#                 'zh': aug_target
#             }

In [ ]:
def get_augment_sentence(words, ooc_word, parser, nlp):
    # Just get the new english sentence first, get translation using baseline separately then combine
    best_sentence = None
    best_score = 0

    # Perform POS tagging
    # Process the ooc word to get its text and POS
    ooc_word_struct = nlp(ooc_word)[0]
    tagged_words = pos_tag(words)
    same_pos_words_indices = [i for i, (word, pos) in enumerate(tagged_words) if pos == ooc_word_struct.tag_]

    for pos in tqdm(range(len(same_pos_words_indices))):
        original_word = words[pos]
        words[pos] = ooc_word  # Modify the list in place
        nbest_list = parser.parse(words, rerank=False)

        if len(nbest_list) > 0:
            # Get the best parse according to the reranker
            best_parse = nbest_list.get_parser_best()
            parser_score = abs(best_parse.parser_score)
            # Update the best sentence if this parse is better than the current best
            if parser_score > best_score:
                best_score = parser_score
                best_sentence = " ".join(words)

        words[pos] = original_word  # Revert the modification

    return best_sentence

In [ ]:
def augment_dataset_and_save(ooc_words, K=20):
    data_dir = "./data"
    splits = ["train"]
    dataset = load_dataset("iwslt2017", "iwslt2017-en-zh", data_dir=data_dir, cache_dir=data_dir)

    # Initialize the parser
    model_dir = find('models/bllip_wsj_no_aux').path
    parser = RerankingParser.from_unified_model_dir(model_dir)
    nlp = spacy.load('en_core_web_md')

    for split in splits:
        augmented_sentences = []
        len_indices = len(dataset[split])
        for ooc_word in tqdm(ooc_words):
            selected_indices = set()

            # Randomly select max K sentences from inputs to augment
            # For each ooc word, K sentences will be augmented
            while len(selected_indices) < K:
                random_index = random.randint(0, len_indices - 1)
                if random_index in selected_indices:
                    print("Skipping due to index collision")
                    continue

                example = dataset[split][random_index]
                source_sentence = example['translation']['en']

                # Tokenize source sentence
                words = word_tokenize(source_sentence)
                augmented = get_augment_sentence(words, ooc_word, parser, nlp)
                if augmented:
                    augmented_sentences.append(augmented)
                    selected_indices.add(random_index)

        path = os.path.join(data_dir, f"{split}_parse_augmented_en.txt")
        with open(path, 'w', encoding='utf-8') as f:
            for s in augmented_sentences:
                f.write(s + "\n")


        # augmented_sentences = []
        # indices = list(range(len(dataset[split])))

        # # Randomly choose sentences to do OOC word replacement
        # random_indices = random.sample(indices, min(num_sentences_to_augment, len(indices)))

        # for i in tqdm(random_indices):
        #     example = dataset[split][i]
        #     source_sentence = example['translation']['en']
        #     target_sentence = example['translation']['zh']

        #     augmented = get_augment_sentence(source_sentence, ooc_words, parser)
        #     if augmented:
        #         augmented_sentences.append(augmented)

        # path = os.path.join(data_dir, f"{split}_parse_augmented_en.txt")
        # with open(path, 'w', encoding='utf-8') as f:
        #     for s in augmented_sentences:
        #         f.write(s + "\n")

In [ ]:
with open('data/OOC words.txt', 'r') as f:
    # Read each line and create a set of strings
    ooc_words = {line.strip() for line in f}
augment_dataset_and_save(ooc_words)

Streaming output truncated to the last 5000 lines.
100%|██████████| 3/3 [00:00<00:00,  5.62it/s]

100%|██████████| 1/1 [00:00<00:00, 29.24it/s]

0it [00:00, ?it/s]

100%|██████████| 8/8 [00:14<00:00,  1.82s/it]

100%|██████████| 3/3 [00:03<00:00,  1.07s/it]

100%|██████████| 1/1 [00:00<00:00, 13.70it/s]

100%|██████████| 7/7 [00:06<00:00,  1.08it/s]

0it [00:00, ?it/s]

100%|██████████| 2/2 [00:00<00:00,  5.09it/s]

100%|██████████| 4/4 [00:01<00:00,  2.59it/s]

100%|██████████| 7/7 [00:10<00:00,  1.56s/it]

100%|██████████| 2/2 [00:00<00:00,  2.88it/s]

100%|██████████| 6/6 [00:12<00:00,  2.02s/it]

100%|██████████| 2/2 [00:01<00:00,  1.72it/s]

100%|██████████| 2/2 [00:04<00:00,  2.07s/it]

 16%|█▌        | 8/50 [09:44<44:09, 63.07s/it]
0it [00:00, ?it/s]

100%|██████████| 6/6 [00:11<00:00,  1.93s/it]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

100%|██████████| 1/1 [00:02<00:00,  2.17s/it]

0it [00:00, ?it/s

In [10]:
# Save as jsonlines for translation
import json

def convert_to_jsonl(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as f:
        sentences = f.readlines()

    with open(output_file, 'w', encoding='utf-8') as out_f:
        for sentence in sentences:
            # Remove any leading/trailing whitespace
            sentence = sentence.strip()
            if sentence:  # Ensure the sentence is not empty
                json_object = {
                    "translation": {
                        "en": sentence,
                        "zh": ""
                    }
                }
                # Write the JSON object to the file with a newline character
                out_f.write(json.dumps(json_object) + '\n')

convert_to_jsonl('data/train_parse_augmented_en.txt', 'data/augmented.json')

In [4]:
!rm -r transformers
!git clone https://github.com/huggingface/transformers.git
!cd transformers
!pip install -q ./transformers

Cloning into 'transformers'...
remote: Enumerating objects: 195041, done.
remote: Counting objects: 100% (236/236), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 195041 (delta 107), reused 155 (delta 62), pack-reused 194805
Receiving objects: 100% (195041/195041), 207.77 MiB | 13.25 MiB/s, done.
Resolving deltas: 100% (138221/138221), done.
Updating files: 100% (4221/4221), done.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [5]:
%cd /content/drive/MyDrive/project
!pip install -q -r requirements.txt

/content/drive/MyDrive/project
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 28.1 MB/s eta

In [6]:
# Translate using pretrained baseline model
!python ft.py \
    --model_name_or_path Helsinki-NLP/opus-mt-en-zh \
    --do_predict \
    --source_lang en \
    --target_lang zh \
    --max_source_length 512 \
    --test_file ./data/augmented.json \
    --validation_file ./data/validation.json \
    --output_dir ./opus \
    --per_device_train_batch_size=4 \
    --per_device_eval_batch_size=4 \
    --predict_with_generate

2024-04-16 15:41:01.526595: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-16 15:41:01.526646: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-16 15:41:01.632595: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-16 15:41:03.911551: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
04/16/2024 15:41:06 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
04/16/2024 15:41:06 - INFO - __main__ - Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gp

In [3]:
import json

def create_augmented_json(existing_train, augment_en, augment_zh, output_file):
    combined_data = []
    with open(existing_train, 'r', encoding='utf-8') as existing_train_f:
        for line in existing_train_f:
            data = json.loads(line.strip())
            combined_data.append(data)

    with open(augment_en, 'r', encoding='utf-8') as augment_en_f, \
         open(augment_zh, 'r', encoding='utf-8') as augment_zh_f:
        english_sentences = [line.strip() for line in augment_en_f]
        chinese_translations = [line.strip() for line in augment_zh_f]
        for en, zh in zip(english_sentences, chinese_translations):
            json_object = {
                "translation": {
                    "en": en,
                    "zh": zh
                }
            }
            combined_data.append(json_object)


    # Open the existing train JSON file to append new augmented data
    with open(output_file, 'w', encoding='utf-8') as out_file:
        for item in combined_data:
            out_file.write(json.dumps(item) + '\n')


# Example usage
create_augmented_json('data/train.json', 'data/train_parse_augmented_en.txt', 'opus/generated_predictions.txt', 'data/train_augmented.json')